<a href="https://colab.research.google.com/github/Johnkideys/melanoma-CNN-project/blob/main/cnn_models_for_melanoma_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to connect to my google drive
import os
from google.colab import drive

# mount google drive to google colab session
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


### ResNet - Configuration 4

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

# paths to data
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Data_oversampled"
val_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Validation_Data"
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Draft"


# initial parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-4

# creating funvtions to load dataset and apply data augmenttaion
def augment_data(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

def load_dataset(directory, batch_size=BATCH_SIZE, img_size=IMG_SIZE, shuffle=True, augment=False):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        image_size=img_size,
        batch_size=batch_size,
        label_mode='binary',
        shuffle=shuffle
    )

    if augment: # can choose to augment or not when loading the daatset
        dataset = dataset.map(augment_data)

    return dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Datasets are created here for train and validation
train_dataset = load_dataset(train_data_dir, augment=True)
val_dataset = load_dataset(val_data_dir, shuffle=False)

# ResNet50 model initialisation
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # freezing the model first, fine tuning later

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# class weights created, I will assign 3 and 1
class_weight = {0: 3.0, 1: 1.0}
# Higher weight for melanoma class

# Compiling the model with weighted loss
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Training stage below
callbacks = [
    ModelCheckpoint(os.path.join(checkpoint_dir, "resnet_isic_2017_ver4.h5"), save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
]

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    class_weight=class_weight,
    callbacks=callbacks
)

## FINE-TUNING (Unfreeze Some ResNet50 Layers)
base_model.trainable = True
for layer in base_model.layers[:100]:  # Keep first 100 layers frozen # So last 75 layers are fine tuned
    layer.trainable = False

# Recompile the model
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning is better
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fine-tune applied
fine_tune_epochs = 10
history_fine = model.fit(
    train_dataset,
    epochs=fine_tune_epochs,
    validation_data=val_dataset,
    class_weight=class_weight,
    callbacks=callbacks
)

### Evaluation of the model
y_true = []
for _, labels in val_dataset:
    y_true.append(labels.numpy())
y_true = np.concatenate(y_true)

y_pred_probs = model.predict(val_dataset)
y_pred = (y_pred_probs > 0.5).astype(int)

melanoma_f1 = f1_score(y_true, y_pred, pos_label=0)
melanoma_precision = precision_score(y_true, y_pred, pos_label=0)
melanoma_recall = recall_score(y_true, y_pred, pos_label=0)
accuracy = accuracy_score(y_true, y_pred)

print(f"Final Validation Metrics (Melanoma - Class 0):")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score - (Melanoma): {melanoma_f1:.4f}")
print(f"Precision - (Melanoma): {melanoma_precision:.4f}")
print(f"Recall (Melanoma): {melanoma_recall:.4f}")

# Plot charts to understand behaviour as epochs progress
def plot_training_history(history, title="Model Training"):
    plt.figure(figsize=(12, 5))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'{title} - Loss')
    plt.legend()

    plt.show()

plot_training_history(history, title="Initial Training")
plot_training_history(history_fine, title="Fine-Tuning")

# Classification report printed
print("Classification Report (Melanoma Focused):")
print(classification_report(y_true, y_pred, target_names=['Melanoma', 'Non-Melanoma']))


In [ ]:
# In this cell I load the test data and use it on the trained model
# Path to test data on my drive
test_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Test_Sorted_Data"

# Load the test dataset - no augmentation is used for test set
test_dataset = load_dataset(test_data_dir, augment=False, shuffle=False)

# evaluation of model on test set
y_true_test = []
for _, labels in test_dataset:
    y_true_test.append(labels.numpy())
y_true_test = np.concatenate(y_true_test)

# Predict the test dataset
y_pred_probs_test = model.predict(test_dataset)
y_pred_test = (y_pred_probs_test > 0.5).astype(int)

# Getting performance metrics for the test set
test_melanoma_f1 = f1_score(y_true_test, y_pred_test, pos_label=0)
test_melanoma_precision = precision_score(y_true_test, y_pred_test, pos_label=0)
test_melanoma_recall = recall_score(y_true_test, y_pred_test, pos_label=0)
test_accuracy = accuracy_score(y_true_test, y_pred_test)

print(f"\nTest Set Metrics (Melanoma - Class 0):")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score (Melanoma): {test_melanoma_f1:.4f}")
print(f"Precision (Melanoma): {test_melanoma_precision:.4f}")
print(f"Recall (Melanoma): {test_melanoma_recall:.4f}")

# Classification report provided
print("\nClassification Report on Test Set (Melanoma Focused):")
print(classification_report(y_true_test, y_pred_test, target_names=['Melanoma', 'Non-Melanoma']))


### ResNet - Configuration 6

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Parameters
IMG_SIZE = (128, 128)  # Resize all of the images to this(square size)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 1

# Setting random seeds for reproducibility purposes
import random
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Paths to datasets on my google drive
training_data_path = '/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Data'
validation_data_path = '/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Validation_Data'

# Preparing Data Generators (slower method, initially used)
train_datagen = ImageDataGenerator(
   rescale=1./255,  # Normalise pixel values
   rotation_range=30,
   width_shift_range=0.2,
   height_shift_range=0.2,
   shear_range=0.2,
   zoom_range=0.2,
   horizontal_flip=True
)

val_datagen = ImageDataGenerator(
   rescale=1./255  # I am only normalising validation data, no augmentation
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
   training_data_path,
   target_size=IMG_SIZE,
   batch_size=BATCH_SIZE,
   class_mode='binary'
)

# Validation data generator
val_generator = val_datagen.flow_from_directory(
   validation_data_path,
   target_size=IMG_SIZE,
   batch_size=BATCH_SIZE,
   class_mode='binary'
)

# Getting balanced class weights
class_weights = compute_class_weight(
   class_weight='balanced',
   classes=np.unique(train_generator.classes),
   y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# ResNet Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Freeze the base model layers to keep pretrained weights
base_model.trainable = False

# Add custom classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Replace Flatten with GlobalAveragePooling
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularisation of 0.5 for dropout
output = Dense(1, activation='sigmoid')(x)

# model created below
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
             loss='binary_crossentropy',  # For binary classification
             metrics=['accuracy'])

## Train the model, takes time cause of ImageDataGenerator probably
history = model.fit(
   train_generator,
   epochs=EPOCHS,
   validation_data=val_generator,
   class_weight=class_weights
)

## Saving model to new directory on drive
model.save('/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Main/isic_resnet_model_ver6.h5')

# Evaluate model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Plot charts over epochs
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot charts for  loss over epochs
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# other metrics
from sklearn.metrics import classification_report
y_true = val_generator.classes
y_pred = (model.predict(val_generator) > 0.5).astype("int32")
print(classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys()))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Loading the saved model to use on test set
model_path = '/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Main/isic_resnet_model_ver6.h5'
model = load_model(model_path)

# Test data path
test_data_path = '/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Test_Sorted_Data'

# Getting test data generator ready
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
   test_data_path,
   target_size=(128, 128),
   batch_size=32,
   class_mode='binary',
   shuffle=False
)

# Predictions..
y_pred_proba = model.predict(test_generator)
y_pred = (y_pred_proba > 0.5).astype("int32")
y_true = test_generator.classes

# Classification report given
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

### DenseNet

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

#
# To get same results every time I run it, using "random" library
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# issues connecxting to drive so pasted code again here
from google.colab import drive
drive.mount('/content/drive')

# directories in drive
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Data"
val_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Validation_Data"
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Main"
os.makedirs(checkpoint_dir, exist_ok=True)

# initial parameters given
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE = 1e-4

# Function to augment images created as was confusing before if needed
def augment_image(image, label):
    """augmentation function"""
    # Random transformations part
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    # Random rotation part
    angle = tf.random.uniform([], -0.52, 0.52)  # -30 to 30 degrees
    image = tf.image.rot90(image, k=tf.cast(angle/(np.pi/2), tf.int32) % 4)
    # Random zoom and crop part
    image = tf.image.central_crop(image, central_fraction=0.8)
    image = tf.image.resize(image, IMG_SIZE)

    return tf.clip_by_value(image, 0.0, 1.0), label

# Load the dataset
def create_dataset(data_dir, batch_size, shuffle=True, augment=False):
    """Create tf.data.Dataset from the directory"""
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        image_size=IMG_SIZE,
        batch_size=batch_size,
        label_mode='binary',
        shuffle=shuffle,
        seed=SEED
    )

    # Normalise and will try augment and without
    dataset = dataset.map(lambda x, y: (x/255.0, y))
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)

    return dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Creating the  datasets
train_dataset = create_dataset(train_data_dir, BATCH_SIZE, shuffle=True, augment=True)
val_dataset = create_dataset(val_data_dir, BATCH_SIZE, shuffle=False)

# Class weights creation using balanced
labels = np.concatenate([y.numpy() for x, y in train_dataset], axis=0).flatten()
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))
print(f"Class weights: {class_weights}")

# F1 score
"""
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()
"""

# DenseNet201 model initialised
base_model = DenseNet201(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMG_SIZE, 3)
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(*IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

# Compiling the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', F1Score(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Callbacks for the model
callbacks = [
    ModelCheckpoint(
        os.path.join(checkpoint_dir, 'best_model.h5'),
        monitor='val_f1_score',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_f1_score',
        patience=5,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_f1_score',
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        verbose=1
    )
]

print("Starting initial training...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)
# This section is for fine-tuning
print("\nUnfreezing layers for fine-tuning...")
base_model.trainable = True
for layer in base_model.layers[:-100]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', F1Score(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print("\nStarting fine-tuning...")
history_fine = model.fit(
    train_dataset,
    epochs=EPOCHS//2,
    initial_epoch=history.epoch[-1],
    validation_data=val_dataset,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# Evaluation of the model
def evaluate_model(dataset, name):
    y_true = np.concatenate([y.numpy() for x, y in dataset], axis=0).flatten()
    y_pred = (model.predict(dataset) > 0.5).astype(int)

    print(f"\n{name} Set Evaluation:")
    print(classification_report(y_true, y_pred, target_names=['Non-Melanoma', 'Melanoma']))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print(f"\nF1 Score: {f1_score(y_true, y_pred):.4f}")

evaluate_model(train_dataset, "Training")
evaluate_model(val_dataset, "Validation")

# Saved final model to checkpoints directory in drive
final_model_path = os.path.join(checkpoint_dir, 'densenet_model_ver1.h5')
model.save(final_model_path)
print(f"\nFinal model saved to: {final_model_path}")

In [ ]:
# cell to use saved densenet model on test set of ISIC 2017 sorted
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# paths to test set
test_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Test_Sorted_Data"
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
SEED = 42

# loading the model saved
model_path = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Main/densenet_model_ver1.h5"
model = tf.keras.models.load_model(
    model_path,
    custom_objects={
        'F1Score': F1Score
    }
)

# function to get the test set
def create_test_dataset(data_dir, batch_size):
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        image_size=IMG_SIZE,
        batch_size=batch_size,
        label_mode='binary',
        shuffle=False,
        seed=SEED
    )
    dataset = dataset.map(lambda x, y: (x / 255.0, y))  # Normalsi
    return dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

test_dataset = create_test_dataset(test_data_dir, BATCH_SIZE)

# Evaluate model and print metrics
y_true = np.concatenate([y.numpy() for x, y in test_dataset], axis=0).flatten()
y_pred_probs = model.predict(test_dataset)
y_pred = (y_pred_probs > 0.5).astype(int)

print("Test Set Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Melanoma', 'Melanoma']))
print("\n Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print(f"\nF1 Score: {f1_score(y_true, y_pred):.4f}")


### MobileNetV2 - Configuration 2


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

# paths to data
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Data"
val_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Validation_Data"
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Main"

# initial parameters, 96 96 for speeding up training
IMG_SIZE = (96, 96)
BATCH_SIZE = 16
EPOCHS = 15
LEARNING_RATE = 1e-4

# Setting random seeds for reproducibility purposes
import random
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

os.makedirs(checkpoint_dir, exist_ok=True)

# Data generators created and images normalised
datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = datagen.flow_from_directory(
    val_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# For Class Imbalance we use class weights
labels = train_generator.classes  # Get labels
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))
print(f"Class Weights: {class_weights}")

# MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # Freeze the base model initially

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)  # Sigmoid activation used for binary classification

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# callbacks created including early stopping
callbacks = [
    ModelCheckpoint(os.path.join(checkpoint_dir, "mobilenet_isic_2017_ver2.h5"), save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    class_weight=class_weights,  # Apply class weights
    callbacks=callbacks
)

# Fine Tuning section
print("\nFine-tuning top layers of MobileNetV2...")

# Unfreeze last few layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Keep most of the layers frozen
    layer.trainable = False

# Recompile with a lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE / 10),  #
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_fine_tune = model.fit(
    train_generator,
    epochs=EPOCHS // 2,  # Less epochs for fine-tuning
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks
)

# Evaluate the model
y_true = val_generator.classes  # real labels of data
y_pred_probs = model.predict(val_generator)  # Predicted probabilities
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert to binary predictions

# Compute metrics
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
accuracy = np.mean(y_true == y_pred)

print(f"\nFinal Validation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Melanoma', 'Melanoma']))


In [ ]:
# print(classification_report(y_true, y_pred, target_names=['Non-Melanoma', 'Melanoma']))
# Checked below for data quality
import os

# Paths
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Data"
val_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Validation_Data"

# Function to count images in subfolders
def count_images(directory):
    counts = {}
    for subfolder in os.listdir(directory):
        subfolder_path = os.path.join(directory, subfolder)
        if os.path.isdir(subfolder_path):
            counts[subfolder] = len(os.listdir(subfolder_path))
    return counts

# Get counts for train and validation datasets
train_counts = count_images(train_data_dir)
val_counts = count_images(val_data_dir)
print("\nTraining Data numbers:")
for category, count in train_counts.items():
    print(f"{category}: {count} images")

print("\nValidation Data numbers:")
for category, count in val_counts.items():
    print(f"{category}: {count} images")


In [ ]:
# Using the trained model on the test set

import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

# loading model with parameters
test_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Test_Sorted_Data"
IMG_SIZE = (96, 96)
BATCH_SIZE = 16
model_path = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Main/mobilenet_isic_2017_ver2.h5"
model = tf.keras.models.load_model(model_path)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Getting the predicitions
y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Computing operformance metrics
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

print(f"\nTest Set Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Classification report:
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))


### MobileNetV2 - Configuration 3.1

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

# paths to data
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Data"
val_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Validation_Data"
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints"

# initial parameters, using 224 224 for more detail
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE = 1e-4


# Data Augmentation steps applied
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = ImageDataGenerator(rescale=1.0 / 255.0).flow_from_directory(
    val_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Class weights using balanced
labels = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))
print(f"Class Weights: {class_weights}")

# Focal loss function created for class imbalance
import tensorflow.keras.backend as K

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        return -K.mean(alpha * K.pow(1 - pt, gamma) * K.log(pt))
    return loss

# MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # Freeze the base model initially

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(1, activation='sigmoid')(x)  # Sigmoid for the binary classification

model = Model(inputs=base_model.input, outputs=output)

# Compile model but with focal loss
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss=focal_loss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

C#allbacks listed
callbacks = [
    ModelCheckpoint(os.path.join(checkpoint_dir, "mobilenet_isic_2017.h5"), save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
]

# Training the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks
)

# Fine-tuning the model
print("\nFine-tuning deeper layers of MobileNetV2...")
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=focal_loss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

history_fine_tune = model.fit(
    train_generator,
    epochs=EPOCHS // 2,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks
)

# Evaluation of model
y_true = val_generator.classes
y_pred_probs = model.predict(val_generator)
y_pred = (y_pred_probs > 0.5).astype(int)

f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
accuracy = np.mean(y_true == y_pred)

print(f"\nFinal Validation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Melanoma', 'Non-Melanoma']))

# Training charts plotted
def plot_history(history):
    plt.figure(figsize=(12, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Plot initial training history
plot_history(history)

# Plot fine-tuning history
plot_history(history_fine_tune)


In [ ]:
# Cell for predictions on test set
# Loading test set from drive
test_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Test_Sorted_Data"

test_dataset = create_dataset(test_data_dir, BATCH_SIZE, shuffle=False)

# Evaluation and printing of metrics
y_test_true = np.concatenate([y.numpy() for _, y in test_dataset], axis=0)
y_test_probs = model.predict(test_dataset)
y_test_pred = (y_test_probs > 0.5).astype(int)

test_f1 = f1_score(y_test_true, y_test_pred)
test_precision = precision_score(y_test_true, y_test_pred)
test_recall = recall_score(y_test_true, y_test_pred)
test_accuracy = np.mean(y_test_true == y_test_pred)

print(f"\nTest Metrics:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")

print("\nTest Classification Report:")
print(classification_report(y_test_true, y_test_pred, target_names=['Melanoma', 'Non-Melanoma']))


### MobileNetV2 - Configuration 5

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

# Mount Google Drive in this cell for ease
from google.colab import drive
drive.mount('/content/drive')

# Set random seeds for reproducibility purposes
import random
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


# paths to data
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Data"
val_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Sorted_Validation_Data"
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Checkpoints/Main"

# initial parameters, using 224 224 for more detail
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE = 1e-4



# function to apply data augmentation
def augment_image(image, label):
    """
    This function applies data augmentation to an image from the dataset.
    """
    # horizontal flip
    image = tf.image.random_flip_left_right(image)
    # Random rotation
    angle = tf.random.uniform([], -0.52, 0.52)  # -30 to 30 degrees in radians
    image = tf.image.rot90(image, k=tf.cast(angle / (np.pi / 2), tf.int32) % 4)
    # Random brightness adjustments
    image = tf.image.random_brightness(image, max_delta=0.2)
    # Random contrast adjustment
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    # Random zoom and crop
    image = tf.image.central_crop(image, central_fraction=0.8)
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

# Loading dataset
def create_dataset(data_dir, batch_size, shuffle=True, augment=False):
    """
    Create a tf.data.Dataset from a directory of images.
    """
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        image_size=IMG_SIZE,
        batch_size=batch_size,
        label_mode='binary',
        shuffle=shuffle
    )

    # Normalise images to between 0 and 1
    dataset = dataset.map(lambda x, y: (x / 255.0, y))

    # Apply augmentation if needed
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Prefetch for faster loading of images
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

# Create training dataset with the augmentation
train_dataset = create_dataset(train_data_dir, BATCH_SIZE, shuffle=True, augment=True)
# Create validation dataset without the augmentation
val_dataset = create_dataset(val_data_dir, BATCH_SIZE, shuffle=False)

# class weights created here
labels = np.concatenate([y.numpy() for x, y in train_dataset], axis=0)
labels = labels.flatten()  # Flatten the 2D array to 1D
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))
print(f"Class Weights: {class_weights}")

#Focal loss function
import tensorflow.keras.backend as K

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        return -K.mean(alpha * K.pow(1 - pt, gamma) * K.log(pt))
    return loss

# MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # Freeze the base model initially

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

model = Model(inputs=base_model.input, outputs=output)

# Compile model with the Focal Loss
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss=focal_loss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

# callbacks listed
callbacks = [
    ModelCheckpoint(os.path.join(checkpoint_dir, "mobilenet_isic_2017_ver5.h5"), save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
]

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    class_weight=class_weights,
    callbacks=callbacks
)

# Fine-Tuning stage
print("\nFine-tuning deeper layers of MobileNetV2...")

# Unfreeze last 50 layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Recompile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=focal_loss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

history_fine_tune = model.fit(
    train_dataset,
    epochs=EPOCHS // 2,
    validation_data=val_dataset,
    class_weight=class_weights,
    callbacks=callbacks
)

# Evaluation and predictions
y_true = np.concatenate([y.numpy() for x, y in val_dataset], axis=0)
y_true = y_true.flatten()  # Flatten the 2D array to 1D
y_pred_probs = model.predict(val_dataset)
y_pred = (y_pred_probs > 0.5).astype(int)

# Compute metrics
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
accuracy = np.mean(y_true == y_pred)

print(f"\nFinal Validation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Print classification report
print("\nCorrected Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Melanoma', 'Non-Melanoma']))

In [ ]:
# ------Load Model from Checkpoint-----
# Load the best model weights from the checkpoint
best_model_path = os.path.join(checkpoint_dir, "mobilenet_isic_2017_ver5.h5")
model.load_weights(best_model_path)
#print(f"Loaded best model weights from: {best_model_path}")

# Loading test data
test_data_dir = "/content/drive/MyDrive/Colab Notebooks/Keele - MSc project/Data/ISIC-2017_Test_Sorted_Data"

# Create test dataset
test_dataset = create_dataset(test_data_dir, BATCH_SIZE, shuffle=False)

# Evaluation
# Get true labels from the test dataset
y_true_test = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)
y_true_test = y_true_test.flatten()  # Flattening to 1D array

# predictions
y_pred_probs_test = model.predict(test_dataset)

# Convert probabilities to predictions using 0.5 as threshold
y_pred_test = (y_pred_probs_test > 0.5).astype(int)

# Compute metrics
test_accuracy = np.mean(y_true_test == y_pred_test)
test_f1 = f1_score(y_true_test, y_pred_test)
test_precision = precision_score(y_true_test, y_pred_test)
test_recall = recall_score(y_true_test, y_pred_test)
print("\nTest Set Metrics:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")

# Classification report for the test set given
print("\nTest Set Classification Report:")
print(classification_report(y_true_test, y_pred_test, target_names=['Melanoma', 'Non-Melanoma']))